In [13]:
from elasticsearch import Elasticsearch

elastic_id = "elastic"

elastic_pw = "4dnlOhB4hWeOX*fR0NjA"

client = Elasticsearch(
    "http://192.168.0.249:9200",
    basic_auth=("elastic", "your_pw"),
    request_timeout=60,          # 🔧 timeout 늘리기
    retry_on_timeout=True,
    max_retries=3
)


client.info()

ObjectApiResponse({'name': 'user1-Legion-Y540-15IRH', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'kTHWtNOERMGMyzb6Tg_4ow', 'version': {'number': '9.0.4', 'build_flavor': 'default', 'build_type': 'deb', 'build_hash': 'e03b6699309ed1458e07dbcd94918b36f5ffee9e', 'build_date': '2025-07-16T22:08:05.257395109Z', 'build_snapshot': False, 'lucene_version': '10.1.0', 'minimum_wire_compatibility_version': '8.18.0', 'minimum_index_compatibility_version': '8.0.0'}, 'tagline': 'You Know, for Search'})

In [15]:
client.indices.create(index="sex-index", ignore=400)

# 2. 문서 삽입
doc = {
    "name": "홍길동",
    "message": "면접자 응답 예시입니다",
    "timestamp": "2025-07-22T14:00:00"
}
client.index(index="demo-index", document=doc)

C:\Users\main\AppData\Local\Temp\ipykernel_12996\1376453694.py:1: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  client.indices.create(index="sex-index", ignore=400)


ObjectApiResponse({'_index': 'demo-index', '_id': 'qWXINZgBsPJCz4hqxVvY', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 2})

In [50]:
import pandas as pd

import json

import os
import json
import pandas as pd

base_dir = r"TL_01.Management_Female_Experienced"
all_records = []


for root, _, files in os.walk(base_dir):
    for file in files:
        if file.endswith(".json"):
            json_path = os.path.join(root, file)
            try:
                with open(json_path, "r", encoding="utf-8") as f:
                    data = json.load(f)

                info = data["dataSet"]["info"]
                question = data["dataSet"]["question"]
                answer = data["dataSet"]["answer"]
                raw_info = data["rawDataInfo"]

                # Emotion/Intent 안전하게 꺼내기
                emotion_list = answer.get("emotion", [])
                intent_list = answer.get("intent", [])

                first_emotion = emotion_list[0] if len(emotion_list) > 0 else {}
                first_intent = intent_list[0] if len(intent_list) > 0 else {}

                record = {
                    "date": data["dataSet"]["info"]["date"],
                    "occupation": data["dataSet"]["info"]["occupation"],
                    "channel": data["dataSet"]["info"]["channel"],
                    "place": data["dataSet"]["info"]["place"],
                    "gender": data["dataSet"]["info"]["gender"],
                    "ageRange": data["dataSet"]["info"]["ageRange"],
                    "experience": data["dataSet"]["info"]["experience"],

                    "question_text": data["dataSet"]["question"]["raw"]["text"],
                    "question_word_count": data["dataSet"]["question"]["raw"]["wordCount"],

                    "answer_text": data["dataSet"]["answer"]["raw"]["text"],
                    "answer_word_count": data["dataSet"]["answer"]["raw"]["wordCount"],
                    
                    "summary": data["dataSet"]["answer"]["summary"]["text"],
                    "summary_word_count": data["dataSet"]["answer"]["summary"]["wordCount"],

                    # 감정이 존재할 경우만 추출 (없으면 None)
                    "emotion_expression": data["dataSet"]["answer"]["emotion"][0]["expression"] if data["dataSet"]["answer"]["emotion"] else None,
                    "emotion_category": data["dataSet"]["answer"]["emotion"][0]["category"] if data["dataSet"]["answer"]["emotion"] else None,

                    "intent_expression": data["dataSet"]["answer"]["intent"][0]["expression"] if data["dataSet"]["answer"]["intent"] else None,
                    "intent_category": data["dataSet"]["answer"]["intent"][0]["category"] if data["dataSet"]["answer"]["intent"] else None,

                    "question_audio_path": data["rawDataInfo"]["question"]["audioPath"],
                    "answer_audio_path": data["rawDataInfo"]["answer"]["audioPath"]
                }

                all_records.append(record)

            except Exception as e:
                print(f"⚠️ {file} 처리 중 오류 발생: {e}")

df = pd.DataFrame(all_records)
print(df.head())


       date occupation channel   place  gender ageRange   experience  \
0  20230116         BM    MOCK  ONLINE  FEMALE    35-44  EXPERIENCED   
1  20230116         BM    MOCK  ONLINE  FEMALE    35-44  EXPERIENCED   
2  20230116         BM    MOCK  ONLINE  FEMALE    35-44  EXPERIENCED   
3  20230116         BM    MOCK  ONLINE  FEMALE    35-44  EXPERIENCED   
4  20230116         BM    MOCK  ONLINE  FEMALE    45-54  EXPERIENCED   

                                       question_text  question_word_count  \
0                  요즘 논쟁이 많은 우버 택시 찬반에 대해서 논해 보시겠습니까                    9   
1                         지금 지원자분이 제일 가고 싶은 회사가 어딜까요                    7   
2  면접자님은 이전 직장을 그만둔 특별한 이유가 있습니까 그리고 지금 다니는 직장도 같...                   22   
3  요즘 공부하고 있는 분야가 있다면 무엇인지 한번 말씀을 해 보시고요 또 그동안에 본...                   21   
4  일을 하게 되면 중국으로 출장을 갈 일이 조금 생길 수도 있습니다 그 빈도수도 굉장...                   20   

                                         answer_text  answer_word_count  \
0  우버 택시에 대해서 말씀드려보겠습니다. 우버 택

In [ ]:
df_valid_emotion = df[(df['emotion_category'] !=    'None')].copy()

df_valid_emotion.head()

,date,occupation,channel,place,gender,ageRange,experience,question_text,question_word_count,answer_text,answer_word_count,summary,summary_word_count,emotion_expression,emotion_category,intent_expression,intent_category,question_audio_path,answer_audio_path
0,20230116,BM,MOCK,ONLINE,FEMALE,35-44,EXPERIENCED,요즘 논쟁이 많은 우버 택시 찬반에 대해서 논해 보시겠습니까,9,우버 택시에 대해서 말씀드려보겠습니다. 우버 택시 이제 바뀌는 현대사회에서 당연히 ...,140,우버 택시는 현대사회에서 당연히 나타날 부분이고 더 발전된 부분이 나타날 것이라 생...,45,None,None,,attitude,/Mock/01.Management/Female/Experienced/ckmk_q_...,/Mock/01.Management/Female/Experienced/ckmk_a_...
1,20230116,BM,MOCK,ONLINE,FEMALE,35-44,EXPERIENCED,지금 지원자분이 제일 가고 싶은 회사가 어딜까요,7,음 지금 제가 본인이 원했을 때 제가 가고 싶은 회사의 종류가 몇 가지가 있을 것 ...,158,"저는 돈을 많이 주는 회사, 딱딱하고 엄격한 분위기 회사, 부드럽고 웃음이 많은 회...",38,None,None,,attitude,/Mock/01.Management/Female/Experienced/ckmk_q_...,/Mock/01.Management/Female/Experienced/ckmk_a_...
2,20230116,BM,MOCK,ONLINE,FEMALE,35-44,EXPERIENCED,면접자님은 이전 직장을 그만둔 특별한 이유가 있습니까 그리고 지금 다니는 직장도 같...,22,아 이전 직장을 그만둔 계기에 대해서 말씀드리겠습니다. 어 이전 직장을 가장 그만두...,150,이전 직장을 그만둔 가장 큰 계기는 매너리즘이었습니다. 매너리즘에 빠지다 보니 몸이...,43,None,None,,attitude,/Mock/01.Management/Female/Experienced/ckmk_q_...,/Mock/01.Management/Female/Experienced/ckmk_a_...
3,20230116,BM,MOCK,ONLINE,FEMALE,35-44,EXPERIENCED,요즘 공부하고 있는 분야가 있다면 무엇인지 한번 말씀을 해 보시고요 또 그동안에 본...,21,네 요즘 공부하고 있는 것에 대해서 말씀드리겠습니다. 어 이제 경영 파트로 오게 되...,160,"경영 파트로 오게 되면서 경영 회사의 원리를 알아야 겠다는 생각을 했고, 그러다 보...",50,None,None,,attitude,/Mock/01.Management/Female/Experienced/ckmk_q_...,/Mock/01.Management/Female/Experienced/ckmk_a_...
4,20230116,BM,MOCK,ONLINE,FEMALE,45-54,EXPERIENCED,일을 하게 되면 중국으로 출장을 갈 일이 조금 생길 수도 있습니다 그 빈도수도 굉장...,20,당연히 해야죠. 저는 어 전에 했던 사업으로 인해서 중국에서의 경험이 조금 많습니다...,80,이 회사가 중국에 사업을 넓히는 것에 관심이 있다면 제 경험을 살려서 노하우를 공유...,25,None,None,,attitude,/Mock/01.Management/Female/Experienced/ckmk_q_...,/Mock/01.Management/Female/Experienced/ckmk_a_...


In [48]:
import os
import json
import pandas as pd

def load_all_json_to_df(json_dir):
    records = []

    for root, _, files in os.walk(json_dir):
        for file in files:
            if file.endswith('.json'):
                json_path = os.path.join(root, file)
                try:
                    with open(json_path, 'r', encoding='utf-8') as f:
                        data = json.load(f)

                    info = data["dataSet"]["info"]
                    question = data["dataSet"]["question"]["raw"]
                    answer = data["dataSet"]["answer"]["raw"]
                    summary = data["dataSet"]["answer"].get("summary", {}).get("text", "")
                    emotions = data["dataSet"]["answer"].get("emotion", [])
                    intents = data["dataSet"]["answer"].get("intent", [])

                    audio_path = data["rawDataInfo"]["answer"].get("audioPath", None)

                    emotion_expr = " | ".join([e["expression"] for e in emotions if "expression" in e])
                    emotion_cat = " | ".join([e["category"] for e in emotions if "category" in e])
                    intent_expr = " | ".join([i["expression"] for i in intents if "expression" in i])
                    intent_cat = " | ".join([i["category"] for i in intents if "category" in i])

                    records.append({
                        "date": info.get("date"),
                        "occupation": info.get("occupation"),
                        "channel": info.get("channel"),
                        "place": info.get("place"),
                        "gender": info.get("gender"),
                        "ageRange": info.get("ageRange"),
                        "experience": info.get("experience"),
                        "question_text": question.get("text", ""),
                        "question_word_count": question.get("wordCount", 0),
                        "answer_text": answer.get("text", ""),
                        "answer_word_count": answer.get("wordCount", 0),
                        "summary": summary,
                        "emotion_expression": emotion_expr,
                        "emotion_category": emotion_cat,
                        "intent_expression": intent_expr,
                        "intent_category": intent_cat,
                        "audio_path": audio_path,
                        "json_file": file  # Optional: 추적용
                    })
                except Exception as e:
                    print(f"⚠️ Failed to process {json_path}: {e}")

    return pd.DataFrame(records)


In [49]:
df = load_all_json_to_df("TL_01.Management_Female_Experienced")
print(df.head())

       date occupation channel   place  gender ageRange   experience  \
0  20230116         BM    MOCK  ONLINE  FEMALE    35-44  EXPERIENCED   
1  20230116         BM    MOCK  ONLINE  FEMALE    35-44  EXPERIENCED   
2  20230116         BM    MOCK  ONLINE  FEMALE    35-44  EXPERIENCED   
3  20230116         BM    MOCK  ONLINE  FEMALE    35-44  EXPERIENCED   
4  20230116         BM    MOCK  ONLINE  FEMALE    45-54  EXPERIENCED   

                                       question_text  question_word_count  \
0                  요즘 논쟁이 많은 우버 택시 찬반에 대해서 논해 보시겠습니까                    9   
1                         지금 지원자분이 제일 가고 싶은 회사가 어딜까요                    7   
2  면접자님은 이전 직장을 그만둔 특별한 이유가 있습니까 그리고 지금 다니는 직장도 같...                   22   
3  요즘 공부하고 있는 분야가 있다면 무엇인지 한번 말씀을 해 보시고요 또 그동안에 본...                   21   
4  일을 하게 되면 중국으로 출장을 갈 일이 조금 생길 수도 있습니다 그 빈도수도 굉장...                   20   

                                         answer_text  answer_word_count  \
0  우버 택시에 대해서 말씀드려보겠습니다. 우버 택

In [46]:
df['emotion_category'].unique()

array(['', 'positive', 'neutral', 'positive | neutral',
       'positive | positive | positive', 'negative',
       'positive | neutral | positive', 'neutral | neutral',
       'positive | neutral | positive | neutral', 'positive | positive',
       'neutral | positive', 'neutral | neutral | neutral | neutral',
       'neutral | negative', 'positive | positive | positive | positive',
       'neutral | neutral | neutral', 'negative | negative',
       'negative | positive',
       'positive | positive | positive | positive | positive | positive',
       'neutral | positive | neutral'], dtype=object)

In [47]:
df_emotion = df[df['emotion_category'].str.strip() != ''].copy()


df_emotion

,date,occupation,channel,place,gender,ageRange,experience,question_text,question_word_count,answer_text,answer_word_count,summary,emotion_expression,emotion_category,intent_expression,intent_category,audio_path,json_file
12,20230116,BM,MOCK,ONLINE,FEMALE,-34,EXPERIENCED,이루고 싶은 인생의 목표가 있으세요 있으시면 왜 그것을 이루고 싶은 지에 대해서도 ...,15,저는 일단은 당분간은 회사 생활을 열심히 해서 제 돈을 좀 모으고 싶고요. 제 삶의...,79,"저는 당분간은 회사 생활을 열심히 해서 돈을 모으고 싶으며, 제 삶의 큰 목표는 반...",p-interest,positive,,attitude,/Mock/01.Management/Female/Experienced/ckmk_a_...,ckmk_d_bm_f_e_102974.json
26,20230116,BM,MOCK,ONLINE,FEMALE,45-54,EXPERIENCED,본인께서 어 살아오시면서 가장 힘들었던 경험은 무엇이며 그 경험을 어떻게 극복했는지...,13,예전에 언어를 배우기 위하여 중국에서 생활했었던 적이 있었습니다. 제게 그때 가장 ...,52,"언어를 배우기 위해 중국에서 생활할 때, 음식에 적응하는 것이 가장 힘들었습니다. ...",u-fact,neutral,c_private | c_private,background | background,/Mock/01.Management/Female/Experienced/ckmk_a_...,ckmk_d_bm_f_e_105083.json
28,20230116,BM,MOCK,ONLINE,FEMALE,45-54,EXPERIENCED,면접자분 지금까지 살아오시면서 새로운 환경에서 전혀 몰랐던 일을 맡아서 한 경험이 ...,15,네 전혀 몰랐던 일을 새로운 환경에서 맡아본 경험이 있습니다. 정말 저에게는 신세계...,57,새로운 환경에서 안전 요원 일을 맡았던 경험은 저에게는 신세계 같은 일이었고 너무도...,p-happiness | u-fact,positive | neutral,,attitude,/Mock/01.Management/Female/Experienced/ckmk_a_...,ckmk_d_bm_f_e_105085.json
33,20230116,BM,MOCK,ONLINE,FEMALE,45-54,EXPERIENCED,사람들이 대부분 전공에 맞춰서 일을 찾게 되는데요 혹시 전공에 관련해서 책은 부족하...,20,전공 관련하여 저는 캐드 같은 것들이 저에게 부족하다고 생각합니다. 하지만 지금의 ...,62,전공 관련 캐드 같은 것들이 부족하지만 접근하기 쉬운 소프트웨어를 이용하여 캐드와 ...,p-interest,positive,,attitude,/Mock/01.Management/Female/Experienced/ckmk_a_...,ckmk_d_bm_f_e_105092.json
35,20230116,BM,MOCK,ONLINE,FEMALE,35-44,EXPERIENCED,지원자께서는 스트레스를 받으면 보통 어떻게 하시는가요 그리고 왜 그런 방법으로 스트...,14,저는 스트레스를 받을 때 사람들을 만나면 스트레스가 풀리는 것 같습니다. 저는 기본...,138,저는 스트레스를 받을 때 사람들을 만나 이야기를 하고 공감을 받고 하고 싶은 이야기...,p-affection | p-affection | p-happiness,positive | positive | positive,,attitude,/Mock/01.Management/Female/Experienced/ckmk_a_...,ckmk_d_bm_f_e_107834.json
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
984,20230116,BM,MOCK,ONLINE,FEMALE,-34,EXPERIENCED,회사 업무에는 인사 업무라는 것이 있습니다 그렇다면 인사 업무 중 아는 것이 있다면...,30,제가 인사 업무를 담당하지는 않았지만 기본적인 내용은 알고 있습니다. 일단 연봉제라...,140,많은 직장인이 활용하는 연봉제는 일 년 동안의 연봉에 따라 그 사람의 역량을 평가해...,u-fact | u-fact | u-fact | u-fact,neutral | neutral | neutral | neutral,,attitude,/Mock/01.Management/Female/Experienced/ckmk_a_...,ckmk_d_bm_f_e_82941.json
990,20230116,BM,MOCK,ONLINE,FEMALE,-34,EXPERIENCED,최근에 본 뉴스들 가운데서 가장 인상 깊었던 뉴스가 있으실까요 어 그 뉴스가 무엇인...,18,저는 최근에 금리 인상과 관련된 이슈를 인상 깊게 봤습니다. 최근에 글로벌적으로 모...,121,최근에 금리 인상과 관련된 이슈를 인상 깊게 봤습니다. 미국이 물가를 잡기 위해서 ...,u-fact,neutral,,attitude,/Mock/01.Management/Female/Experienced/ckmk_a_...,ckmk_d_bm_f_e_84709.json
1014,20230116,BM,MOCK,ONLINE,FEMALE,-34,EXPERIENCED,요즘 고령화 사회로 가면서 치매로 인한 문제가 많이 대두되고 있는데 건보가 하는 일...,17,고령화 사회 치매 문제를 개선하기 위한 방안으로는 세 가지 정도를 말씀드리고 싶습니...,110,고령화 사회 치매 문제의 개선 방안으로는 노인 주간 보호센터 이용시간 및 이용 횟수...,u-belief,neutral,m_critical,attitude,/Mock/01.Management/Female/Experienced/ckmk_a_...,ckmk_d_bm_f_e_96246.json
1019,20230116,BM,MOCK,ONLINE,FEMALE,-34,EXPERIENCED,귀하께서는 회사의 인사 업무 중 아는 것이 있다면 구체적으로 그것에 대해 이야기해 ...,25,인사 업무는 최고 경영자와 직원 사이에 연결 다리 역할을 하는 것이 인사 업무라고 ...,122,조직 관리에서 구성원을 평가하여 인사 정보를 획득하고 활용하는 작업은 가장 기본적인...,u-fact | u-fact | u-fact,neutral | neutral | neutral,,attitude,/Mock/01.Management/Female/Experienced/ckmk_a_...,ckmk_d_bm_f_e_96835.json


In [5]:
import os
import numpy as np
import librosa
import pandas as pd

# 피처 추출 함수
def extract_features(filepath):
    y, sr = librosa.load(filepath, sr=None)  # 원래 sampling rate 유지
    pitches, magnitudes = librosa.piptrack(y=y, sr=sr)
    pitch_series = [np.mean(p[p > 0]) if np.any(p > 0) else 0 for p in pitches.T]
    energy_series = librosa.feature.rms(y=y)[0]

    return {
        "filename": os.path.basename(filepath),
        "avg_pitch": float(np.mean(pitch_series)),
        "pitch_std": float(np.std(pitch_series)),
        "pitch_slope": float(np.polyfit(range(len(pitch_series)), pitch_series, 1)[0]),

        "avg_energy": float(np.mean(energy_series)),
        "energy_std": float(np.std(energy_series)),
        "energy_slope": float(np.polyfit(range(len(energy_series)), energy_series, 1)[0]),
    }

# 폴더 내 모든 wav 파일 처리 - "ckmk_a"로 시작하는 파일만
def process_audio_folder(folder_path):
    data = []

    for fname in os.listdir(folder_path):
        if fname.endswith(".wav") and fname.startswith("ckmk_a"):
            fpath = os.path.join(folder_path, fname)
            try:
                features = extract_features(fpath)
                data.append(features)
            except Exception as e:
                print(f"⚠️ {fname} 처리 중 오류: {e}")

    return pd.DataFrame(data)

# 예시 실행
folder = r"C:\Users\main\Documents\emotionCLF\Mock\01.Management\Female\Experienced"
df = process_audio_folder(folder)

# 결과 확인
print(df.head())


                   filename   avg_pitch   pitch_std  pitch_slope  avg_energy  \
0  ckmk_a_bm_f_e_101795.wav  822.964948  723.983401    -0.008916    0.055558   
1  ckmk_a_bm_f_e_101796.wav  794.939154  709.146651     0.028108    0.052824   
2  ckmk_a_bm_f_e_101799.wav  787.692153  701.780618     0.040217    0.056065   
3  ckmk_a_bm_f_e_101801.wav  771.414216  693.385413     0.036848    0.057125   
4  ckmk_a_bm_f_e_102773.wav  956.700485  556.414458    -0.072225    0.044644   

   energy_std  energy_slope  
0    0.054843     -0.000002  
1    0.055384     -0.000001  
2    0.058963      0.000003  
3    0.056853      0.000001  
4    0.045908     -0.000009  


In [ ]:
df.to_csv("cal")


,filename,avg_pitch,pitch_std,pitch_slope,avg_energy,energy_std,energy_slope
0,ckmk_a_bm_f_e_101795.wav,822.964948,723.983401,-0.008916,0.055558,0.054843,-0.000002
1,ckmk_a_bm_f_e_101796.wav,794.939154,709.146651,0.028108,0.052824,0.055384,-0.000001
2,ckmk_a_bm_f_e_101799.wav,787.692153,701.780618,0.040217,0.056065,0.058963,0.000003
3,ckmk_a_bm_f_e_101801.wav,771.414216,693.385413,0.036848,0.057125,0.056853,0.000001
4,ckmk_a_bm_f_e_102773.wav,956.700485,556.414458,-0.072225,0.044644,0.045908,-0.000009
...,...,...,...,...,...,...,...
1026,ckmk_a_bm_f_e_98828.wav,926.655536,658.556168,0.009640,0.040474,0.036991,0.000004
1027,ckmk_a_bm_f_e_98829.wav,1137.146767,758.693565,0.044609,0.040747,0.036338,0.000010
1028,ckmk_a_bm_f_e_98830.wav,1080.071010,685.237123,0.013359,0.039754,0.037688,0.000002
1029,ckmk_a_bm_f_e_98831.wav,1120.189868,677.048548,-0.003631,0.037463,0.038831,0.000003


In [4]:
filtered_df

,filename,avg_pitch,pitch_std,pitch_slope,avg_energy,energy_std,energy_slope
0,ckmk_a_bm_f_e_101795.wav,822.964948,723.983401,-0.008916,0.055558,0.054843,-0.000002
1,ckmk_a_bm_f_e_101796.wav,794.939154,709.146651,0.028108,0.052824,0.055384,-0.000001
2,ckmk_a_bm_f_e_101799.wav,787.692153,701.780618,0.040217,0.056065,0.058963,0.000003
3,ckmk_a_bm_f_e_101801.wav,771.414216,693.385413,0.036848,0.057125,0.056853,0.000001
4,ckmk_a_bm_f_e_102773.wav,956.700485,556.414458,-0.072225,0.044644,0.045908,-0.000009
...,...,...,...,...,...,...,...
1026,ckmk_a_bm_f_e_98828.wav,926.655536,658.556168,0.009640,0.040474,0.036991,0.000004
1027,ckmk_a_bm_f_e_98829.wav,1137.146767,758.693565,0.044609,0.040747,0.036338,0.000010
1028,ckmk_a_bm_f_e_98830.wav,1080.071010,685.237123,0.013359,0.039754,0.037688,0.000002
1029,ckmk_a_bm_f_e_98831.wav,1120.189868,677.048548,-0.003631,0.037463,0.038831,0.000003
